In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_matching/chinese/main')

In [2]:
%tensorflow_version 2.x
!pip install tensorflow-addons

In [3]:
import csv
import tensorflow as tf
import numpy as np
import pprint
import logging
import time
import math

from tensorflow_addons.optimizers.cyclical_learning_rate import Triangular2CyclicalLearningRate

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.2.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Enabled: True


In [0]:
def get_vocab(f_path):
  k2v = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip()
      k2v[line] = i
  return k2v

In [0]:
# stream data from text files
def data_generator(f_path, char2idx):
  with open(f_path) as f:
    print('Reading', f_path)
    for i, line in enumerate(csv.reader(f, delimiter=',')):
      if i == 0:
        continue
      text1, text2, label = line
      text1 = [char2idx.get(c, len(char2idx)) for c in list(text1)]
      text2 = [char2idx.get(c, len(char2idx)) for c in list(text2)]
      yield ((text1, text2), int(label))


def dataset(is_training, params):
  _shapes = (([None], [None]), ())
  _types = ((tf.int32, tf.int32), tf.int32)
  _pads = ((0, 0), -1)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_path'], params['char2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_path'], params['char2idx']),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
  
  return ds

In [0]:
class RE2(tf.keras.Model):
  def __init__(self, params: dict):
    super().__init__()
    
    self.pretrained_embedding = tf.Variable(np.load(params['embedding_path']), name='pretrained_embedding', dtype=tf.float32)
    self.embed_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.birnn = [tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        params['hidden_size'], return_sequences=True), name='birnn_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.enc_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.align_t = self.add_weight(name='temperature',
                                   shape=(),
                                   trainable=True,
                                   initializer=tf.initializers.constant(math.sqrt(1/params['hidden_size'])))
    self.align_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.align_fc1 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='align_fc1_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.align_fc2 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='align_fc2_%d'%(i+1)) for i in range(params['num_blocks'])]
      
    self.fusion_fc1 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='fusion_fc1_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.fusion_fc2 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='fusion_fc2_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.fusion_fc3 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='fusion_fc3_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.fusion_fc4 = [tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='fusion_fc4_%d'%(i+1)) for i in range(params['num_blocks'])]
    self.fusion_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.out_drop1 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_fc = tf.keras.layers.Dense(params['hidden_size'], params['activation'], name='out_fc')
    self.out_drop2 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear = tf.keras.layers.Dense(1, name='out_linear')
    
  
  
  def call(self, inputs, training=False):
    x1, x2 = inputs
    
    if x1.dtype != tf.int32:
      x1 = tf.cast(x1, tf.int32)
    if x2.dtype != tf.int32:
      x2 = tf.cast(x2, tf.int32)
    
    batch_sz = tf.shape(x1)[0]
    
    mask1 = tf.sign(x1)
    mask2 = tf.sign(x2)
    
    x1 = self.embedding(x1, training=training)
    x2 = self.embedding(x2, training=training)
    
    res_x1, res_x2 = x1, x2
    
    for i in range(params['num_blocks']):
      
      if i > 0:
        x1 = self.connection(x1, res_x1, i)
        x2 = self.connection(x2, res_x2, i)
        res_x1, res_x2 = x1, x2
    
      x1_enc = self.encoding(x1, mask1, i, training=training)
      x2_enc = self.encoding(x2, mask2, i, training=training)
    
      x1 = tf.concat((x1, x1_enc), -1)
      x2 = tf.concat((x2, x2_enc), -1)
      
      align_1, align_2 = self.alignment(x1, x2, mask1, mask2, i, training=training)
      
      x1 = self.fusion(x1, align_1, i, training=training)
      x2 = self.fusion(x2, align_2, i, training=training)
    
    x1 = self.pooling(x1, mask1)
    x2 = self.pooling(x2, mask2)
    
    x = self.prediction(x1, x2, training=training)
    
    return x
  
  
  def embedding(self, x, training):
    x = tf.nn.embedding_lookup(self.pretrained_embedding, x)
    x = self.embed_dropout(x, training=training)
    return x
  
  
  def connection(self, x, res, i):
    if i == 1:
      x = tf.concat((res, x), -1)
    elif i > 1:
      hidden_size = x.shape[-1]
      x = (res[:, :, -hidden_size:] + x) * tf.math.sqrt(0.5)
      x = tf.concat((res[:, :, :-hidden_size], x), -1)
    return x
    
    
  def encoding(self, x, mask, i, training):
    mask = tf.cast(tf.expand_dims(mask, -1), tf.float32)
    x *= mask
    x = self.birnn[i](x)
    x = self.enc_dropout(x, training=training)
    return x
  
  
  def alignment(self, x1, x2, mask1, mask2, i, training):
    mask1 = tf.cast(tf.expand_dims(mask1, -1), tf.float32)
    mask2 = tf.cast(tf.expand_dims(mask2, -1), tf.float32)
    
    x1_ = self.align_fc1[i](self.align_dropout(x1, training=training))
    x2_ = self.align_fc2[i](self.align_dropout(x2, training=training))
    align = tf.matmul(x1_, x2_, transpose_b=True) * self.align_t
    mask = tf.matmul(mask1, mask2, transpose_b=True)
    align = mask * align + (1 - mask) * tf.float32.min
    align_1 = tf.nn.softmax(align, 1)
    align_2 = tf.nn.softmax(align, 2)
    
    x2 = tf.matmul(align_1, x1, transpose_a=True)
    x1 = tf.matmul(align_2, x2)
    return x1, x2
  
  
  def fusion(self, x, align, i, training):
    x = tf.concat([self.fusion_fc1[i](tf.concat((x, align), -1)),
                   self.fusion_fc2[i](tf.concat((x, x - align), -1)),
                   self.fusion_fc3[i](tf.concat((x, x * align), -1))], -1)
    x = self.fusion_dropout(x, training=training)
    x = self.fusion_fc4[i](x)
    return x
  
  
  def pooling(self, x, mask):
    mask = tf.cast(tf.expand_dims(mask, -1), tf.float32)
    return tf.reduce_max(x * mask, 1)
  
  
  def prediction(self, x1, x2, training):
    x = tf.concat((x1, x2, x1 * x2, x1 - x2), -1)
    x = self.out_drop1(x, training=training)
    x = self.out_fc(x)
    x = self.out_drop2(x, training=training)
    x = self.out_linear(x)
    x = tf.squeeze(x, 1)
    return x

In [0]:
params = {
  'train_path': '../data/train.csv',
  'test_path': '../data/test.csv',
  'vocab_path': '../vocab/char.txt',
  'embedding_path': '../vocab/char.npy',
  'batch_size': 32,
  'buffer_size': 100000,
  'num_blocks': 2,
  'dropout_rate': 0.2,
  'hidden_size': 300,
  'activation': tf.nn.relu,
  'init_lr': 1e-4,
  'max_lr': 8e-4,
  'label_smooth': .2,
  'clip_norm': 5.,
  'num_patience': 10,
}

In [0]:
def label_smoothing(inputs, epsilon):
  V = inputs.get_shape().as_list()[-1]
  return ((1-epsilon) * inputs) + (epsilon / V)

In [9]:
params['char2idx'] = get_vocab(params['vocab_path'])
params['vocab_size'] = len(params['char2idx']) + 1

model = RE2(params)
model.build([[None, None], [None, None]])
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = Triangular2CyclicalLearningRate(
  initial_learning_rate = params['init_lr'],
  maximal_learning_rate = params['max_lr'],
  step_size = 4 * params['buffer_size'] // params['batch_size'],)
optim = tf.optimizers.Adam(params['init_lr'])
global_step = 0

best_acc = .0
count = 0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)

while True:
  # TRAINING
  for ((text1, text2), labels) in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model((text1, text2), training=True)
      loss = tf.nn.sigmoid_cross_entropy_with_logits(
        labels=label_smoothing(tf.cast(labels, tf.float32), params['label_smooth']),
        logits=logits)
      loss = tf.reduce_mean(loss)
      
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, model.trainable_variables))
    
    if global_step % 100 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1
  
  # EVALUATION
  m = tf.keras.metrics.Accuracy()

  for ((text1, text2), labels) in dataset(is_training=False, params=params):
    logits = tf.sigmoid(model((text1, text2), training=False))
    y_pred = tf.cast(tf.math.greater_equal(logits, .5), tf.int32)
    m.update_state(y_true=labels, y_pred=y_pred)

  acc = m.result().numpy()
  logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))

  if acc > best_acc:
    best_acc = acc
    # you can save model here
    count = 0
  else:
    count += 1
  logger.info("Best Accuracy: {:.3f}".format(best_acc))

  if count == params['num_patience']:
    print(params['num_patience'], "times not improve the best result, therefore stop training")
    break

[('birnn_1/forward_lstm/lstm_cell_1/kernel:0', TensorShape([300, 1200])),
 ('birnn_1/forward_lstm/lstm_cell_1/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('birnn_1/forward_lstm/lstm_cell_1/bias:0', TensorShape([1200])),
 ('birnn_1/backward_lstm/lstm_cell_2/kernel:0', TensorShape([300, 1200])),
 ('birnn_1/backward_lstm/lstm_cell_2/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('birnn_1/backward_lstm/lstm_cell_2/bias:0', TensorShape([1200])),
 ('birnn_2/forward_lstm_1/lstm_cell_4/kernel:0', TensorShape([600, 1200])),
 ('birnn_2/forward_lstm_1/lstm_cell_4/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('birnn_2/forward_lstm_1/lstm_cell_4/bias:0', TensorShape([1200])),
 ('birnn_2/backward_lstm_1/lstm_cell_5/kernel:0', TensorShape([600, 1200])),
 ('birnn_2/backward_lstm_1/lstm_cell_5/recurrent_kernel:0',
  TensorShape([300, 1200])),
 ('birnn_2/backward_lstm_1/lstm_cell_5/bias:0', TensorShape([1200])),
 ('align_fc1_1/kernel:0', TensorShape([900, 300])),
 ('align_fc1_1/bias:0'